In [5]:
import numpy as np
'''This method transforms matrices (A) and (B) into 2D polynomials, performs their multiplication and summation, and then transforms the resulting polynomial (C) back into a matrix.'''

'This method transforms matrices (A) and (B) into 2D polynomials, performs their multiplication and summation, and then transforms the resulting polynomial (C) back into a matrix.'

In [6]:
def Poly1(x):
  N=len(x)
  if N==1:
    return x
  else:
    even=Poly1(x[::2])
    odd=Poly1(x[1::2])
    y=[0]*N
    W=np.exp(-2j * np.pi / N)
    for i in range(N//2):
      y[i]=even[i]+(W**i)*odd[i]
      y[i+N//2]=even[i]-(W**i)*odd[i]
    return y
Poly1([1,2,3,5,6,7,8])

[(32+0j),
 (-2.5060407925650656+1.8726740701278808j),
 (-4+0j),
 (4+0j),
 (-7.493959207434934+8.12732592987212j),
 (-4+0j),
 0]

In [7]:
def Poly2(x):
    N = len(x)
    if N == 1:
        return x
    else:
        pp = Poly2([j[::2] for j in x[::2]])
        pi = Poly2([j[1::2] for j in x[::2]])
        ip = Poly2([j[::2] for j in x[1::2]])
        ii = Poly2([j[1::2] for j in x[1::2]])
        W = np.exp(-2j * np.pi / N)
        y = np.zeros((N, N), dtype=complex)
        for k in range(N//2):
            for j in range(N//2):
                y[k][j] = pp[k][j] + W**j * pi[k][j] + W**k * ip[k][j] + W**(k+j) * ii[k][j]
                y[k][j+N//2] = pp[k][j] - W**j * pi[k][j] + W**k * ip[k][j] - W**(k+j) * ii[k][j]
                y[k+N//2][j] = pp[k][j] + W**j * pi[k][j] - W**k * ip[k][j] - W**(k+j) * ii[k][j]
                y[k+N//2][j+N//2] = pp[k][j] - W**j * pi[k][j] - W**k * ip[k][j] + W**(k+j) * ii[k][j]
        return y

# Example usage:
# Create a sample input array
input_array = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]])

# Compute 2D FFT
result = Poly2(input_array)
print(result)
print("numpy",np.fft.fft2(input_array))
result==np.fft.fft2(input_array)

[[ 1.3600000e+02 +0.j -8.0000000e+00 +8.j -8.0000000e+00 +0.j
  -8.0000000e+00 -8.j]
 [-3.2000000e+01+32.j  0.0000000e+00 +0.j  0.0000000e+00 +0.j
   0.0000000e+00 +0.j]
 [-3.2000000e+01 +0.j  2.4492936e-16 +0.j  0.0000000e+00 +0.j
   1.9915985e-16 +0.j]
 [-3.2000000e+01-32.j  0.0000000e+00 +0.j  0.0000000e+00 +0.j
   0.0000000e+00 +0.j]]
numpy [[136. +0.j  -8. +8.j  -8. +0.j  -8. -8.j]
 [-32.+32.j   0. +0.j   0. +0.j   0. +0.j]
 [-32. +0.j   0. +0.j   0. +0.j   0. +0.j]
 [-32.-32.j   0. +0.j   0. +0.j   0. +0.j]]


array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True, False,  True, False],
       [ True,  True,  True,  True]])

In [8]:
def inverse_Poly2(x):
    N = len(x[0])
    if N == 1:
        return x
    else:
      pp=Poly2([j[::2] for j in x[::2]])
      pi=Poly2([j[1::2] for j in x[::2]])
      ip=Poly2([j[::2] for j in x[1::2]])
      ii=Poly2([j[1::2] for j in x[1::2]])
      W, y = np.exp(2j * np.pi / N), np.zeros((N, N), dtype=complex)
      for k in range(N//2):
        for j in range(N//2):
          y[k][j] = pp[k][j] + (W**j) * pi[k][j] + (W**k) * ip[k][j] + (W**k) * (W**j) * ii[k][j]
          y[k][j+N//2] = pp[k][j] - (W**j) * pi[k][j] + (W**j) * ip[k][j] - (W**k) * (W**j) * ii[k][j]
          y[k+N//2][j] = pp[k][j] + (W**j) * pi[k][j] - (W**k) * ip[k][j] - (W**k) * (W**j) * ii[k][j]
          y[k+N//2][j+N//2] = pp[k][j] - (W**j) * pi[k][j] - (W**k) * ip[k][j] + (W**k) * (W**j) * ii[k][j]
      return y/N
inverse_Poly2(result)/4

array([[ 1.+0.00000000e+00j,  2.+6.40929343e-17j,  3.+0.00000000e+00j,
         8.-4.00000000e+00j],
       [ 5.+2.44929360e-16j,  6.+3.03301105e-16j, 11.+4.00000000e+00j,
         8.+1.86557614e-16j],
       [ 9.+0.00000000e+00j, 10.+6.40929343e-17j, 11.+0.00000000e+00j,
        12.-6.40929343e-17j],
       [13.-2.44929360e-16j, 14.-1.86557614e-16j, 15.-2.44929360e-16j,
        16.-3.03301105e-16j]])

In [9]:
def HL(x):
    N = len(x[0][0])
    if N == 1:
        return [[sum(x[i][0][0] * x[i][1][0] for i in range(len(x)))]]
    else:
      pp = [[x[i][0][::2], x[i][1][::2]] for i in range(len(x))]
      pi = [[x[i][0][::2], x[i][1][1::2]] for i in range(len(x))]
      ip = [[x[i][0][1::2], x[i][1][::2]] for i in range(len(x))]
      ii = [[x[i][0][1::2], x[i][1][1::2]] for i in range(len(x))]
      pp, pi, ip, ii = HL(pp), HL(pi), HL(ip), HL(ii)
      W,y = np.exp(-2j * np.pi / N),np.zeros((N, N), dtype=complex)
      for k in range(N//2):
        for j in range(N//2):
          y[k][j] = pp[k][j] + W**j * pi[k][j] + W**k * ip[k][j] + W**(k+j) * ii[k][j]
          y[k][j+N//2] = pp[k][j] - W**j * pi[k][j] + W**k * ip[k][j] - W**(k+j) * ii[k][j]
          y[k+N//2][j] = pp[k][j] + W**j * pi[k][j] - W**k * ip[k][j] - W**(k+j) * ii[k][j]
          y[k+N//2][j+N//2] = pp[k][j] - W**j * pi[k][j] - W**k * ip[k][j] + W**(k+j) * ii[k][j]
      return y

In [10]:
#A = np.array([[7, 8], [3, 4]])
#B = np.array([[5, 6], [1, 2]])
A = np.array([[1, 10,33,2], [3, 4,34,6],[34,76,87,99],[90,75,45,0]])
B = np.array([[55,67,88, 6], [70,12,32, 8],[23,98,89,11],[21,44,55,66]])
#A = np.array([[1, 2,3,4], [5, 6,7,8],[9,10,11,12],[13,14,15,16]])
#B = np.array([[1, 2,3,4], [5, 6,7,8],[9,10,11,12],[13,14,15,16]])
# Multiply them using np.dot()
print(np.dot(A, B))
def fast_matrix_multiplication(A, B):
    A, B = np.fft.ifft2(A), np.fft.ifft2(B)
    A = np.fft.fft(A, axis=1)
    B = np.fft.fft(B, axis=0)
    x = np.array([[A[:, i], B[i]] for i in range(len(A))])
    return HL(x)
#some rounding error
fast_matrix_multiplication(A,B)

[[ 1556  3509  3455   581]
 [ 1353  3845  3748   820]
 [11270 16072 18612  8303]
 [11235 11340 14325  1635]]


array([[ 1556.+0.00000000e+00j,  3509.+8.96441457e-14j,
         3455.+0.00000000e+00j,   581.-8.96441457e-14j],
       [ 1353.-3.02548992e-13j,  3845.-1.36854280e-13j,
         3748.-3.23827230e-13j,   820.-1.17566093e-13j],
       [11270.+0.00000000e+00j, 16072.+2.37857025e-13j,
        18612.+0.00000000e+00j,  8303.-2.37857025e-13j],
       [11235.+3.02548992e-13j, 11340.+5.26598124e-13j,
        14325.+3.23827230e-13j,  1635.-2.72177751e-13j]])